THE MODEL

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [14]:
import os

data_dir = 'data'
num_classes = len(os.listdir(data_dir))
print(f'Number of classes: {num_classes}')



Number of classes: 8


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))  # Adjusted number of output units

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

# Save the model
model.save('enhanced_cnn_mango_leaf_disease.h5')


Found 3200 images belonging to 8 classes.
Found 798 images belonging to 8 classes.
Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1303s 13s/step - accuracy: 0.5849 - loss: 5.2458 - val_accuracy: 0.1253 - val_loss: 57.4497
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1271s 13s/step - accuracy: 0.8401 - loss: 1.5507 - val_accuracy: 0.1353 - val_loss: 58.4806
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1283s 13s/step - accuracy: 0.8899 - loss: 1.1109 - val_accuracy: 0.3409 - val_loss: 27.8476
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1306s 13s/step - accuracy: 0.8952 - loss: 0.9351 - val_accuracy: 0.3120 - val_loss: 24.9458
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1377s 14s/step - accuracy: 0.9065 - loss: 0.8348 - val_accuracy: 0.5551 - val_loss: 13.5966
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1262s 13s/step - accuracy: 0.9302 - loss: 0.5705 - val_accuracy: 0.4386 - val_loss: 24.4938
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1288s 13s/step - accuracy: 0.9289 - loss: 0.5392 - val_accuracy: 0.7343 - val_loss: 5

In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0, 1]
    return img_array


In [17]:
# Path to your test image
test_image_path = 'ml.jpg'

# Preprocess the image
preprocessed_image = preprocess_image(test_image_path)

# Predict the class
predictions = model.predict(preprocessed_image)
predicted_class = np.argmax(predictions, axis=1)[0]
print(f'Predicted class: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 820ms/step
Predicted class: 5


In [18]:
# List of disease names corresponding to class indices
disease_names = [
    'Anthracnose', 
    'Bacterial Canker',
    'Cutting Weevil',
    'Die Back',
    'Gall Midge',
    'Healthy',
    'Powdery Mildew', 
    'Sooty Mold'
]

# Function to get treatment information based on the disease name
def get_treatment_info(disease_name):
    if disease_name == 'Anthracnose':
        return {
            "drug": "Chlorothalonil or Copper-based fungicides",
            "procedures": "Apply fungicides at 10-15 day intervals, especially during wet seasons. Ensure thorough coverage of the leaves and fruit.",
            "duration": "Continue treatment from flowering until the fruit is harvested."
        }
    elif disease_name == 'Bacterial Canker':
        return {
            "drug": "Copper-based sprays",
            "procedures": "Apply copper sprays during early stages of the disease. Prune infected branches and ensure good air circulation.",
            "duration": "Apply every 7-14 days depending on the severity of the infection."
        }
    elif disease_name == 'Cutting Weevil':
        return {
            "drug": "Chlorpyrifos or Carbaryl",
            "procedures": "Apply insecticides at the base of the tree and affected areas. Monitor for weevil activity and apply as necessary.",
            "duration": "Apply during the weevil's active period, typically in the early growing season."
        }
    elif disease_name == 'Die Back':
        return {
            "drug": "Carbendazim or Copper Oxychloride",
            "procedures": "Apply fungicides to the affected areas. Prune and destroy affected branches.",
            "duration": "Treat immediately upon detection and continue as necessary."
        }
    elif disease_name == 'Gall Midge':
        return {
            "drug": "Imidacloprid or Abamectin",
            "procedures": "Spray insecticides during the early stages of infestation. Ensure proper coverage of the affected areas.",
            "duration": "Apply every 10-14 days during active infestation."
        }
    elif disease_name == 'Healthy':
        return {
            "drug": "None",
            "procedures": "Maintain regular monitoring and good agricultural practices.",
            "duration": "Ongoing."
        }
    elif disease_name == 'Powdery Mildew':
        return {
            "drug": "Sulfur-based fungicides or Potassium bicarbonate",
            "procedures": "Apply fungicides at the first sign of disease. Ensure thorough coverage of leaves.",
            "duration": "Apply every 7-10 days during the growing season."
        }
    elif disease_name == 'Sooty Mold':
        return {
            "drug": "Horticultural oil or Neem oil",
            "procedures": "Apply oil sprays to affected areas. Ensure good coverage and repeat applications as necessary.",
            "duration": "Apply every 7-14 days until the mold is controlled."
        }
    else:
        return {
            "drug": "Unknown",
            "procedures": "No procedures available.",
            "duration": "Unknown."
        }



# Get the disease name for the predicted class
predicted_disease = disease_names[predicted_class]
print(f'The test image is predicted to be: {predicted_disease}')

# Get the treatment information for the predicted disease
treatment_info = get_treatment_info(predicted_disease)
print(f"Best drug to use: {treatment_info['drug']}")
print(f"Best procedures on use: {treatment_info['procedures']}")
print(f"Duration: {treatment_info['duration']}")


The test image is predicted to be: Healthy
Best drug to use: None
Best procedures on use: Maintain regular monitoring and good agricultural practices.
Duration: Ongoing.


In [19]:
model.save('enhanced_cnn_mango_leaf_disease.h5')